<a href="https://colab.research.google.com/github/Duarbert/SAD_DuarteSantos/blob/master/SAD_20160234_20160803.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistemas de Apoio à Decisão ( Run All Working )

#### TPC 5 - PARTE 1 DO TRABALHO FINAL (15 dias)

---------------


#### Com base no seu dataset (Determinado para a sua resolução do Trabalho final)

1. Catalogar o dataset quanto a
    1. Dimensionalidade
    - Esparsidade
    - Resolução
    - Tamanho


2. Catalogar todos as features (Máx 50), quanto a:
    1. valores de valores médios, variâncias,histogramas (crie um módulo que faça isto por si);
    - Tipo (contínuo, discreto ou binário)   
    - Analise NaN's (ou Nulls) e Outliers
    - Analise os resultados
    - Quais não fazem sentido em termos do problema que pretende resolver;
    - Que outras features poderia utilizar para enriquecer a informação do seu problema;
    - Perceba se deve ou não fazer encoding ou one hot encoding para cada feature discreta;
    - Faça uma matriz de correlação entre as features usando ```dataframe.corr()```;



3. Analise em termos estatísticos a Label/target que tem para já como problema.

---

No final, faça um pequeno texto relatando o dataset que está a utilzar, recorrendo à informação que obteve nas perguntas anteriores;

####TPC 6 - PARTE 2 DO TRABALHO FINAL (Último trabalho)

---------------

#### Com base no seu dataset (Determinado para a sua resolução do Trabalho final)

4. Aplicar 2 dos algoritmos dados nas aulas ao vosso dataset, para classificação/regressão/clustering, fazendo:
    1. Fazendo um split training set/test set;
    - Traçando a curva de aprendizagem do método tendo em conta uma modificação sucessiva do tamanho dos dados de treino;
    - Comparando os resultados dos algoritmos e mostrando o que melhor funciona;

***Caso Seja um Regressor***
- Apresentar RMSE dos dois métodos e comparar resultados;
- Rever feautures usadas e procurar melhorar o resultado obtido, testando mais/outras features;
    
***Caso Seja um Classificador***

- Escrever a matriz de confusao resultante da aplicação iterativa do método;
- Medir a sua Precisão e Recall;
- Definir um contexto de uso razoável do algoritmo e perceber se nessa situação é mais relevante a precisão ou o recall;
- De acordo com a métrica encontrada, rever feautures usadas e procurar melhorar o resultado obtido, testando mais/outras features;

#Definição

Este dataset tem como objectivo classificar uma dada instância de um empregado de uma empresa como potêncial candidato a sair da empresa por variadíssimas razões.

*Attrition, in Human Resource terminology, refers to the phenomenon of the employees leaving the company. Attrition in a company is usually measured with a metric called attrition rate, which simply measures the no of employees moving out of the company (voluntary resigning or laid off by the company). Attrition Rate is also referred as churn rate or turnover.*

*High attrition is a cause of concern for a company as it presents a cost to the company. The company loses on the amount it spent to recruit and select these employees and to train them for their respective jobs. The company may also have to spend additional money to fill the vacancies left open by these employees. Hence it becomes critical for a company to keep a tab on the attrition rate which down-sizes the employee base.*

*Attrition rate gives an idea as to how many employees are leaving the company at any given time period. It is an important factor as companies have to prepare to start recruiting for the positions which are critical and cannot be left vacant.*

#Dependências#

In [0]:
#@title Montar a Google Drive em /content/gdrive
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
#@title Importar Módulos

import pandas as pd
import numpy as np
import io
import matplotlib.pyplot as plt

from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

#Carregar os Dados#

In [0]:
#@title Carregar o dataset e definir o índice.
#@markdown Dataframes Gerados:
#@markdown data_orig = Dados originais carregados do CSV.
#@markdown data = Dados para serem mastigados.

# Índice definido em EmployeeNumber
data = pd.DataFrame(pd.read_csv('/content/gdrive/My Drive/SAD/IBMHR.csv', index_col = 'EmployeeNumber'))
print('Criado dataset data para continuar a ser otimizado.')

data_orig = data.copy()
print('\nCriado dataframe data_orig com os dados originais do csv.')
data.set_axis = 1

In [0]:
#@title (Opcional) - O target, Attrition, não está balanceado. Vamos proceder ao undersampling da classe Attrition

# Class count
# value_counts() = Return a Series containing counts of unique values.
#count_class_0, count_class_1 = data.Attrition.value_counts()

# Dividir por classe
#df_class_0 = data[data['Attrition'] == 'No']
#df_class_1 = data[data['Attrition'] == 'Yes']

# Undersampling
# sample() = Return a random sample of items from an axis of object.
#df_class_0_under = df_class_0.sample(count_class_1)
#data = pd.concat([df_class_0_under, df_class_1], axis=0)

#print('Random under-sampling:')
#print(df_class_0_under.Attrition.value_counts())

#data.Attrition.value_counts().plot(kind='bar', title='Count (Attrition)');

#print('\n Target balanceado.')

#Informação do Dataset - Raw

In [0]:
#@title Listar informação head do dataset 
print('!!! A feature EmployeeNumber está a ser considerada como índice, pois não há nem faz sentido haver duplicados nesta feature !!!\n')
display(data.head())

pd.options.display.max_columns = None
data.columns = data.columns.str.strip()
print('\n')
data.info()

#1.(está em 2.2) Catalogar o dataset quanto a#
1. Dimensionalidade
- Esparsidade
- Resolução
- Tamanho

#2.1 Catalogar todos as features (Máx 50), quanto a valores de:#
1. valores médios
- variâncias
- histogramas (crie um módulo que faça isto por si);

In [0]:
#@title Valores Estatísticos
# Easy mode:
# display(data.describe())


# Like a pro
values = [data.mean(), data.var(), data.std()]

result = pd.concat(values,axis=1, join_axes=[data.mean().index])

result.columns = ["Média","Variância","Desvio Padrão"]

result

In [0]:
#@title Histogramas - Gerados for função conforme requisito
#Função para geração dos histogramas.
# Usando a keyword 'All', corre todas as features

def plotHist(datasetVar, columnVar):
    if columnVar == 'All':
      for i in datasetVar.columns:
        if is_numeric_dtype(datasetVar[i]):
          display('You are about to see data from feature ' + i)
          datasetVar.hist(column = i, figsize=[12,4])
          plt.show()
          df3=data[i].groupby([data['Attrition']])
          plt.ylabel('Number of Workers')
          df3.plot(kind='hist', legend = True, title = i, alpha=0.5, figsize=[12,4])
          plt.show()
    else:
        if is_numeric_dtype(datasetVar[columnVar]):
          display('You are about to see data from feature ' + columnVar)
          datasetVar.hist(column = columnVar, figsize=[12,4])
          plt.show()
          df3=data[columnVar].groupby([data['Attrition']])
          plt.ylabel('Number of Workers')
          df3.plot(kind='hist', legend = True, title = columnVar, alpha=0.5, figsize=[12,4])
          plt.show()

plotHist(data,'All') 

#2.2 Tipo (contínuo, discreto ou binário)

In [0]:
#@title Dimensionalidade, Esparsidade, Resolução, Tamanho. Tipo(contínuo, discreto ou binário)

print('Em termos de dimensionalidade, o dataset tem ' + str(len(data.columns)) + ' features / colunas.')
print('\n')

# As features do tipo objecto estão excluídas
for i in data.select_dtypes(exclude='object'):
    print ('\nA esparsidade de valores da feature',i, 'é de:\n', data[i].max()-data[i].min())

print('\nO tamanho(instâcias, features) é de: ' + str(data.shape))
print('\n')

print('Resolução do datset na seguinte tabela:\n')
dataRes = pd.DataFrame(pd.read_csv('/content/gdrive/My Drive/SAD/IBMHR_Features - Catalog.csv'))
display(dataRes)

#2.7 Pré-processamento dos Dados
- Remover feature irrelevantes
- Binarizar de forma básica os valores de Sim e Não
- 2.7  Onde Hot Encoding em features

In [0]:
#@title Binarizar de forma básica, transformando valores de Sim e Não por valores inteiros 1 e 0

mapping = {'Yes':1, 'No':0, 'Female':1, 'Male':0}
data = data.replace(mapping)

print('Yes, e Mulher = 1\nNo, e Male = 0')


In [0]:
#@title  One Hot Encoding com prefixo, pois iria originar features duplicadas

aaa = pd.get_dummies(data.BusinessTravel, prefix = 'BusinessTravel')
bbb = pd.get_dummies(data.Department, prefix = 'Department')
ccc = pd.get_dummies(data.JobRole, prefix = 'JobRole')
ddd = pd.get_dummies(data.MaritalStatus, prefix = 'MaritalStatus')
eee = pd.get_dummies(data.EducationField, prefix = 'EducationField')

values = [data, aaa, bbb, ccc, ddd, eee]

data = pd.concat(values,axis=1, join_axes=[data.index])

# Apagar colunas que foram one hotted

data = data.drop(['BusinessTravel', 'Department', 'JobRole', 'MaritalStatus', 'EducationField'], axis = 1)

In [0]:
#@title Resumo do resultado da binarização básica e one-hot-encoding
data_pp = data
data_pp.info()
data_pp.head()

#2.5 Remover Features Irrelevantes

In [0]:
#@title Remover features irrelevantes

print('As features StandardHours, Over18 e EmployeeCount são consideradas irrelevantes.')
print('\nA feature StandardHours tem sempre o mesmo valor de 8')
print('\nA feature Over18 tem sempre o mesmo valor de True')
print('\nA feature EmployeeCount tem sempre o mesmo valor de 1')

data_pp = data_pp.drop(['StandardHours', 'Over18', 'EmployeeCount'], axis=1)


#2.3 NaNs e Outliers##


In [0]:
#@title Valore Nulos
display('Existem ' + str(data_pp.isnull().sum().sum()) + ' valores nulos no dataset')
print('\n')

In [0]:
#@title Remoção de Outliers com 1º e 3º quantis de 1%
Q1 = data_pp.quantile(0.01)
Q3 = data_pp.quantile(0.99)
IQR = Q3 - Q1

print('A dispersão estatística dos dados tendo em conta quantis min e max de 1% \n\n', IQR)


data_pp_out = data_pp[~((data_pp < (Q1 - 1.5 * IQR)) |(data_pp > (Q3 + 1.5 * IQR))).any(axis=1)]
print('\nOutliers removidos e novo dataset criado com o nome data_pp_out')

#O código abaixo define subsets de dados:
* Considerados sociais e independentes à IBM = data_social
* Relativos a salários = data_salary
* Hisórico na IBM = data_work_hist
* Features gerais relacionadas com a função = data_work
* Features departamentais = data_work_department
* Features de função a desempenhar = data_work_role
* Features de viagens em trabalho = data_work_travel
* Features relativas a educação = data_educ



In [0]:
#@title Definir Subsets

data_social = data_pp_out[['Age','DistanceFromHome','Gender','MaritalStatus_Divorced',
                           'MaritalStatus_Married','MaritalStatus_Single',
                           'RelationshipSatisfaction', 'WorkLifeBalance',
                           'NumCompaniesWorked', 'TotalWorkingYears']].copy()
data_social.info()
print('\n\n')

data_educ = data_pp_out[['Education','EducationField_Human Resources', 'EducationField_Life Sciences',
                         'EducationField_Marketing','EducationField_Medical','EducationField_Other',
                         'EducationField_Technical Degree']].copy()
data_educ.info()
print('\n\n')

data_salary = data_pp_out[['MonthlyIncome', 'DailyRate','HourlyRate','MonthlyRate','PercentSalaryHike']].copy()
data_salary.info()

data_work = data_pp_out[['EnvironmentSatisfaction','JobInvolvement', 'JobLevel',
                         'JobSatisfaction','PerformanceRating','StockOptionLevel',
                         'OverTime']].copy()
data_work.info()
print('\n\n')

data_work_role = data_pp_out[['JobRole_Healthcare Representative','JobRole_Human Resources',
                              'JobRole_Laboratory Technician','JobRole_Manager',
                              'JobRole_Manufacturing Director','JobRole_Research Director',
                              'JobRole_Research Scientist','JobRole_Sales Executive',
                              'JobRole_Sales Representative']].copy()
data_work_role.info()
print('\n\n')

data_work_travel = data_pp_out[['BusinessTravel_Non-Travel', 'BusinessTravel_Travel_Frequently',
                         'BusinessTravel_Travel_Rarely']].copy()
data_work_travel.info()
print('\n\n')

data_work_hist = data_pp_out[['TrainingTimesLastYear', 'YearsAtCompany', 'YearsInCurrentRole',
                              'YearsSinceLastPromotion','YearsWithCurrManager']].copy()
data_work_hist.info()
print('\n\n')

data_work_department = data_pp_out[['Department_Human Resources', 'Department_Research & Development',
                                    'Department_Sales']].copy()

subsets= {'data_social':data_social, 'data_social':data_salary,
          'data_work':data_work, 'data_work_role':data_work_role,
          'data_work_hist':data_work_hist, 
          'data_work_department':data_work_department,
          'data_work_travel':data_work_travel,
          'data_educ': data_educ}


data_work_department.info()

#2.5 À partida os dados atuais fazem sentido para o problema##

Depois de remover as features não desejadas por terem sempre o mesmo valor, não se dascarta à partida nenhuma feature tendo em conta o tipo de classificação e target.
Qualquer feature relacionada com o indíviduo poderá ser relevante.

#2.6 As seguintes features poderiam enriquecer:
- Número de dias com falta por doença
- Número de dias com falta não justificada
- Avaliações de desempenho anuais

#2.8 Matriz de Correlação

In [0]:
#@title
corr = data_pp_out.corr()
corr.style.background_gradient()

#3 Analise em termos estatísticos a Label/target que tem para já como problema

In [0]:
#@title Copiar a coluna de attrition para uma DF temporária

target_stat = data_pp_out.filter(['Attrition'], axis=1)

In [0]:
#@title Tendo em conta que é uma classificação binária, a análise irá se basear na soma e rácio de cada valor.

target_count = data.Attrition.value_counts()
print('False:', target_count[0])
print('True:', target_count[1])
print('Proporção:', round(target_count[0] / target_count[1], 2), ': 1')
plt.ylabel('Nº de Instancias')
plt.xlabel('Soma de 0:False e True:1')

target_count.plot(kind='bar', title='Count (target)');
plt.show()

In [0]:
#@title Esparsidade das features
for i in data_pp_out.select_dtypes(include='int64'):
    print ('O esparsidade de valores da feature ',i, 'é de', data_pp_out[i].max()-data_pp_out[i].min())

#4. Aplicação de Algoritmos

In [0]:
#@title 1.1 Definir o SET, o Target e escolher as features

df_al = data_pp_out.copy()
y = df_al.pop('Attrition')

Opções Subset = 
* data_social; - Feature sociais
* data_educ; - Features de educação
* data_work;  - Features gerais do trabalho
* data_salary;  - Features salariais
* data_work_hist; - Features de histórico interno
* data_work_role; - Features da função na IBM
* data_work_department - Features do departamento
* df_al - Todas as features

In [0]:
#@title Opções de Subset = data_social; data_work; data_salary; data_work_hist ou data_pp_out(todas as features)
#go_set = data_work_department.join(data_work_role.join(data_work))
#go_set = data_social
go_set = data_work
#go set = data_work.join(work_salary)

features = list(go_set.columns[0:49])
X = go_set[features]
go_set.info()

In [0]:
#@title 4.1 Split do dataset, train e test set

X_train, X_test, y_train, y_test = train_test_split(go_set, y, test_size = 0.2, random_state = 5)
print('Features de treino(instâncias, features) ', X_train.shape)
print('Features de teste(instâncias, features) ', X_test.shape)

In [0]:
#@title Função da Matriz Confusão

def get_confusion_matrix(y_true, y_pred):
    n_classes = len(np.unique(y_true))
    conf = np.zeros((n_classes, n_classes))
    for actual, pred in zip(y_true, y_pred):
        conf[int(actual)][int(pred)] += 1
    return conf.astype('int')

![alt text](http://s5047.pcdn.co/wp-content/uploads/2015/10/confusion_matrix.png)



## Decision Tree Classifier

In [0]:
#@title 4.2 Traçando a curva de aprendizagem do método tendo em conta uma modificação sucessiva do tamanho dos dados de treino
score_dt = []
z = 0
k = 0
previous = 0

for i in range(1, 10):
  z = i/10
  X_train, X_test, y_train, y_test = train_test_split(go_set, y, test_size = z, random_state = 5)
  #Decision Tree Classifier
  dt = DecisionTreeClassifier(random_state = 5)
  dt.fit(X_train, y_train)
  res_score = dt.score(X_test, y_test)
  score_dt.append(res_score)
  if(res_score > previous):
    previous = res_score
    k += 0.1

display(go_set.info())
    
score_dt_local = score_dt
print('\n')
plt.plot(score_dt)
plt.ylabel('Score')
plt.xlabel('Tamanho Dados Treino')
plt.show()

In [0]:
#@title Decision Tree Classifier

dt = DecisionTreeClassifier(random_state = 5)
dt.fit(X_train, y_train)

dt_score = dt.score(X_test, y_test)
print('Score de treino = ', dt.score(X_train, y_train))
print('Score de teste = ', dt_score)

# Contribuição das features
rel_feature=pd.Series(dt.feature_importances_,X_test.columns).sort_values(ascending=False)
rel_feature.head(10).plot(kind='bar')
plt.show()

display('A feature que mais contribuí para o resultado do algoritmo é:',rel_feature.head(1))

In [0]:
#@title 7. e 8. Matrix Confusão do Decision Tree
y_preds = dt.predict(X_test)

matrix_dt = confusion_matrix(y_test, y_preds)
print(matrix_dt)

import itertools 
classes = ['No', 'Yes']
# plot confusion matrix
plt.imshow(matrix_dt, interpolation='nearest', cmap=plt.cm.Greens)
plt.title("Confusion Matrix do Decision Tree")
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes)
plt.yticks(tick_marks, classes)

fmt = 'd'
thresh = matrix_dt.max() / 2.
for i, j in itertools.product(range(matrix_dt.shape[0]), range(matrix_dt.shape[1])):
    plt.text(j, i, format(matrix_dt[i, j], fmt),
             horizontalalignment="center",
             color="white" if matrix_dt[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

print('\n', classification_report(y_test, y_preds))

print('\n\n')

In [0]:
# #@title Looping pelos subsets

table_recall = []
table_name = []
table_precision = []
go_set_loop = go_set.copy()

for k in subsets:
  go_set_loop = subsets.get(k)
  features = list(go_set_loop.columns[0:49])
  X = go_set_loop[features]
  
  X_train, X_test, y_train, y_test = train_test_split(go_set, y, test_size = 0.2, random_state = 5)
  print('\n----Vamos ver os dados do subset', k,'----')
  print('\nFeatures de treino(instâncias, features) ', X_train.shape)
  print('\nFeatures de teste(instâncias, features) ', X_test.shape)
  
  loop = DecisionTreeClassifier(random_state = 5)
  loop.fit(X_train, y_train)

  loop_score = loop.score(X_test, y_test)
  print('\nScore de treino = ', loop.score(X_train, y_train))
  print('Score de teste = ', loop_score, '\n')
  
  y_preds = loop.predict(X_test)

  matrix_loop = confusion_matrix(y_test, y_preds)  

  classes = ['No', 'Yes']
  # plot confusion matrix
  plt.imshow(matrix_loop, interpolation='nearest', cmap=plt.cm.Greens)
  plt.title("Confusion Matrix do Decision Tree")
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes)
  plt.yticks(tick_marks, classes)

  fmt = 'd'
  thresh = matrix_loop.max() / 2.
  for i, j in itertools.product(range(matrix_loop.shape[0]), range(matrix_loop.shape[1])):
    plt.text(j, i, format(matrix_loop[i, j], fmt),
             horizontalalignment="center",
             color="blue" if matrix_dt[i, j] > thresh else "red")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()

  print('\n Matriz Confusão\n', matrix_loop)
  print('\n', classification_report(y_test, y_preds))
  table_name.append(k)
  table_precision.append(precision_score(y_test, y_preds))
  table_recall.append(recall_score(y_test, y_preds))
  

df=pd.DataFrame({'Subset': table_name, 'Recall': table_recall, 'Precision': table_precision})

display(df)

##Random Forest Classifier

In [0]:
#@title 4.2 Traçando a curva de aprendizagem do método tendo em conta uma modificação sucessiva do tamanho dos dados de treino

score_rf = []
z = 0
k = 0
previous = 0

for i in range(1, 10):
  z = i/10
  X_train, X_test, y_train, y_test = train_test_split(go_set, y, test_size = z, random_state = 5)
  #Decision Tree Classifier
  rf=RandomForestClassifier(n_estimators=20, max_depth=100, random_state = 5)
  rf.fit(X_train, y_train)
  res_score = rf.score(X_test, y_test)
  score_rf.append(res_score)
  if(res_score > previous):
    previous = res_score
    k += 0.1


score_rf_local = score_rf  
print('\n')
plt.plot(score_rf_local)
plt.ylabel('Score')
plt.xlabel('Tamanho Dados Treino')
plt.show()

In [0]:
#@title Random Forest Classifier

rf = RandomForestClassifier(n_estimators=20, max_depth=100, random_state = 5)
rf.fit(X_train, y_train)

rf_score = rf.score(X_test, y_test)
print('Score de treino = ', rf.score(X_train, y_train))
print('Score de teste = ', rf_score)

# Contribuição das features
rel_feature=pd.Series(rf.feature_importances_,X_test.columns).sort_values(ascending=False)
rel_feature.head(10).plot(kind='bar')
plt.show()

display('A feature que mais contribuí para o resultado do algoritmo é:',rel_feature.head(1))

In [0]:
#@title 7. e 8.  Matrix Confusão do Random Forest
matrix_dt = 0
y_preds = rf.predict(X_test)

matrix_rf = get_confusion_matrix(y_test, y_preds)
print(matrix_rf)

import itertools 
classes = ['No', 'Yes']
# plot confusion matrix
plt.imshow(matrix_rf, interpolation='nearest', cmap=plt.cm.Greens)
plt.title("Confusion Matrix do Decision Tree")
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes)
plt.yticks(tick_marks, classes)

fmt = 'd'
thresh = matrix_rf.max() / 2.
for i, j in itertools.product(range(matrix_rf.shape[0]), range(matrix_rf.shape[1])):
    plt.text(j, i, format(matrix_rf[i, j], fmt),
             horizontalalignment="center",
             color="white" if matrix_rf[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

print('\n', classification_report(y_test, y_preds))

print('\n\n')

In [0]:
#@title Looping pelos subsets

table_recall = []
table_name = []
table_precision = []
go_set_loop = go_set.copy()

for k in subsets:
  go_set_loop = subsets.get(k)
  features = list(go_set_loop.columns[0:49])
  X = go_set_loop[features]
  
  X_train, X_test, y_train, y_test = train_test_split(go_set, y, test_size = 0.2, random_state = 5)
  print('\n----Vamos ver os dados do subset', k,'----')
  print('\nFeatures de treino(instâncias, features) ', X_train.shape)
  print('\nFeatures de teste(instâncias, features) ', X_test.shape)
  
  loop = RandomForestClassifier(n_estimators=20, max_depth=20, random_state = 5)
  loop.fit(X_train, y_train)

  loop_score = loop.score(X_test, y_test)
  print('\nScore de treino = ', loop.score(X_train, y_train))
  print('Score de teste = ', loop_score, '\n')
  
  y_preds = loop.predict(X_test)

  matrix_loop = get_confusion_matrix(y_test, y_preds)  

  classes = ['No', 'Yes']
  # plot confusion matrix
  plt.imshow(matrix_loop, interpolation='nearest', cmap=plt.cm.Greens)
  plt.title("Confusion Matrix do Decision Tree")
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes)
  plt.yticks(tick_marks, classes)

  fmt = 'd'
  thresh = matrix_loop.max() / 2.
  for i, j in itertools.product(range(matrix_loop.shape[0]), range(matrix_loop.shape[1])):
    plt.text(j, i, format(matrix_loop[i, j], fmt),
             horizontalalignment="center",
             color="blue" if matrix_loop[i, j] > thresh else "red")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()

  print('\n Matriz Confusão\n', matrix_loop)
  print('\n', classification_report(y_test, y_preds))
  table_name.append(k)
  table_precision.append(precision_score(y_test, y_preds))
  table_recall.append(recall_score(y_test, y_preds))
  

df=pd.DataFrame({'Subset': table_name, 'Recall': table_recall, 'Precision': table_precision})

display(df)

In [0]:
#@title 4.3 Comparação da curva de aprendizagem dos métodos

plt.plot(score_dt, label='Decision Tree')
plt.plot(score_rf, label='Random Forest')
plt.ylabel('Score')
plt.xlabel('Tamanho Dados Treino')
plt.legend()
plt.show

##K-Nearest-Neighbors

In [0]:
#@title KNN Classifier


kn = KNeighborsClassifier(n_neighbors=5)
kn = kn.fit(X_train, y_train.values)

score_train = kn.score(X_train, y_train)
score_test = kn.score(X_test, y_test)                              

print('Score de treino = ', score_train)
print('Score de teste = ', score_test, '\n')
expected = y_test
predicted = kn.predict(X_test)


# Preparar plot
k_range = range(1, 26)

scores = []

# Loop
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    scores.append(accuracy_score(y_test, predicted))

display(scores)
print('\n')

plt.plot(k_range, scores)
plt.xlabel('Número de Neighbors')
plt.ylabel('Assertividade to Teste')

In [0]:
#@title Perceptron

#Importar o módulo
from sklearn.linear_model import Perceptron

#Fazer o split do treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 5)


#FOR que verifica quais os settings ideais de treino para o perceptron
melhor_n_iter = 0
melhor_eta = 0
melhor_accuracy = 0
melhor_tolerancia = 0
accuracyAtual = 0
z = 0
w = 1

for i in range(20, 25):
  for j in range(1, 10):
    w = 1
    z = j/100
    for k in range(1,10):
      ppnTemp = Perceptron(max_iter=i, eta0=z, random_state=1, tol=0.1/w)
      ppnTemp.fit(X_train, y_train)
      y_pred = ppnTemp.predict(X_test)
      accuracyAtual = accuracy_score(y_test, y_pred)
      if(accuracyAtual > melhor_accuracy):
        melhor_accuracy = accuracyAtual
        melhor_n_iter = i
        melhor_eta = z
        melhor_tolerancia = 0.1/w
      w = w*10
    j += 1
  i += 1


print('melhor accuracy: ')
print(melhor_accuracy)
print('com o numero iteracoes:')
print(melhor_n_iter)
print('melhor multiplicador:')
print(melhor_eta)
print('e tolerancia:')
print(melhor_tolerancia)


#Criar um perceptron com as definições ideais
ppn = Perceptron(max_iter=melhor_n_iter, eta0=melhor_eta, random_state=5, tol=melhor_tolerancia)

# Treinar o perceptron
ppn.fit(X_train, y_train)

#Predizer resultados com dados de teste
y_pred = ppn.predict(X_test)

#Comparação de dados
#Dados inferidos pelo Perceptron em testes
print('Dados inferidos perceptron:')
print(y_pred)

#Dados reais
print('Dados reais:')
print(y_test)

# 9.Definir um contexto de uso razoável do algoritmo e perceber se nessa situação é mais relevante a precisão ou o recall

Neste caso não é relevante obter resultados muito precisos, o ideal é ter um bom balanço entre a precision e o recall.

Apenas os dataset data_work e data_social obtêm este balanço tal como é visível no loop da confusion matrix dos dois algoritmos.

# 10. De acordo com a métrica encontrada, rever feautures usadas e procurar melhorar o resultado obtido, testando mais/outras features;

O dataset com 59 features foi subdividido por áreas e o melhor resultado obtido é somente a conjugação entre os dois datasets que melhor balançam a precision e o recall.